   1  : 1.58,
   0.1: 2.71,
  0.05: 3.84, 
  0.01: 6.63,
 0.005: 7.88, 
 0.001: 10.83

In [35]:
# 2.71 - 1.13

In [2]:
import pandas as pd
import numpy as np
from joblib import dump

from chi import square as cs

In [19]:
data = pd.read_excel("komentar/data_uji_1255_bersih.xlsx")#  data_cleanU1
# data = data[:100]
komentar = data.komentar.tolist()
label = data.label.tolist()

# data.tail()

# data = pd.read_excel("komentar/data_uji_1255_bersih.xlsx")
# komentar += data.komentar.tolist()
# label += data.label.tolist()

y = np.array(label)
print(len(y))
data

300


,Unnamed: 0,Unnamed: 1,komentar,label
0,482.0,175,theme line exo bts 1d cameron dallas jb zayn m...,non spam
1,468.0,268,case custom 25k pakai gambar apa suka foto kar...,spam
2,988.0,136,pak eko hormat belah istri sah udah liat jadi ...,non spam
3,1142.0,478,thenz ka berkat pakai produk gemuk badan aku j...,spam
4,181.0,405,v vonny vebby m marshanda y yasmin l luthya p ...,non spam
...,...,...,...,...
295,407.0,424,siapa blg mslh blg mslah cma ksih sran aja alx...,non spam
296,384.0,250,sllu ajar usaha tahan kualitas costumer neng s...,spam
297,1126.0,103,aasi ane o a uda nenain odu yan sana eui eecay...,spam
298,632.0,16,awal ny ga percaya aku coba nyata hasil cepet ...,spam


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()#
X = vectorizer.fit_transform(komentar)
save = vectorizer.fit(komentar)
dump(save, "model/tfidf.w")
# print(vectorizer.get_feature_names())
fitur = vectorizer.get_feature_names()
print(X.shape)

(300, 3435)


In [21]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

# Ekstraksi dan Seleksi Fitur

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector
          
# import the algorithm you want to evaluate on your features.
from sklearn.naive_bayes import MultinomialNB

# create the SequentialFeatureSelector object, and configure the parameters.
efs = SequentialFeatureSelector(MultinomialNB(), 
           k_features=1500, 
           forward=True, 
           floating=False,
           scoring='accuracy',
           cv=5)

print("&")
# fit the object to the training data.
efs = efs.fit(X, y)
print("selesai")

&


In [ ]:
# print the selected features.
# selected_features = x_train.columns[list(efs.k_feature_idx_)]
# print(selected_features)

# # print the final prediction score.
# print(efs.k_score_)

# # transform our data to the newly selected features.
# x_train_sfs = efs.transform(x_train)
# x_test_sfs = efs.transform(x_test)

In [10]:
efs.k_feature_idx_

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [12]:
index_fitur_terbaik = efs.k_feature_idx_

In [15]:
np.array(fitur)[list(index_fitur_terbaik)]

array(['00', '000iphone', '000oppo', '000samsung', '000vivo', '000xiaomi',
       '0223', '0388', '0540', '080'], dtype='<U35')

In [3]:
# sf = cs.ChiSquare(alpha = 0.1)
# sf.find_best_features(komentar, label, fitur = None)

In [4]:
# print("jumlah fitur terbaik:",len(sf.best_features),"dari",len(sf.fitur), "fitur")
# best_features = sf.best_features

jumlah fitur terbaik: 2749 dari 15136 fitur


# Pemodelan

## TF-IDF

In [5]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = TfidfVectorizer(vocabulary = best_features)#
# X = vectorizer.fit_transform(komentar)
# save = vectorizer.fit(komentar)
# dump(save, "model/tfidf.w")
# # print(vectorizer.get_feature_names())
# fitur = vectorizer.get_feature_names()
# print(X.shape)

(5047, 2749)


## Pemodelan MNB 

In [6]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

# Pemodelan dan Validasi Model

In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X, y)

start = 0.001
stop = 1
step = start
alpha_list = np.arange(start,stop+step,step)
# print(alpha_list)
akurasi = list()

ix = 0 #just counter
for alpha in alpha_list:
    clf.set_params(alpha=alpha)
    temp_akurasi = list()
    for train_index, test_index in skf.split(X, y):
    #     print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        #latih model
        clf.fit(X_train,y_train)
        
        #prediksi
        y_pred = clf.predict(X_test)
        temp_akurasi.append(accuracy_score(y_test, y_pred))
    rerata_akurasi = sum(temp_akurasi)/len(temp_akurasi)
    akurasi.append(rerata_akurasi)
#     print('alpha   :',alpha)
#     print('     akurasi :',rerata_akurasi)
#     print("="*100)
    ix +=1
    if ix%100 !=0:
        print(".", end="")
    else:
#         print(alpha, ' | ', rerata_akurasi)
        print("")

        pass

ditt = {
    "alpha":alpha_list,
    "akurasi":akurasi
}
index = akurasi.index(max(akurasi))
acc = pd.DataFrame.from_dict(ditt)
acc

...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................


,alpha,akurasi
0,0.001,0.977022
1,0.002,0.977616
2,0.003,0.977418
3,0.004,0.977418
4,0.005,0.977418
...,...,...
995,0.996,0.975438
996,0.997,0.975438
997,0.998,0.975438
998,0.999,0.975438


In [8]:
print(alpha_list[index])
print(akurasi[index])

0.002
0.9776162973440201


In [9]:
print(alpha_list[index])
print(akurasi[index])

0.002
0.9776162973440201


# Menyimpan Model

In [10]:
clf.set_params(alpha=alpha_list[index])
clf.fit(X, y)
dump(clf, "model/mnb.m")

['model/mnb.m']

In [15]:
# p = clf.predict(X)

In [30]:
# ditt = {
#     "Prediksi":p,
#     "label":label,
#     "komentar":komentar
# }
# # index = akurasi.index(max(akurasi))
# akk = pd.DataFrame.from_dict(ditt)

In [31]:
# akk.to_excel("coba_.xlsx")